In [1]:
import psycopg2
import pandas as pd
import numpy as np
import re
from datetime import datetime
import pytz
import re
from operator import itemgetter

pd.set_option('display.max_columns', 999)

VALID_FEATURES = [
    'pair_id','open_time','close_time','dow','tod',
    'open','high','low','close',
    'number_of_trades','volume','quote_asset_volume','taker_buy_base_asset_volume','taker_buy_quote_asset_volume',
    'ma14','ma30','ma90',
    'sup14','sup30','sup90',
    'res14','res30','res90',
    'atr','atr_diff','atr_ma14',
    'rsi','rsi_diff','rsi_ma14',
    'trend_up','trend_up3','trend_up14','trend_up30',
    'cs_ss','cs_ssr','cs_hm','cs_hmr','cs_brh','cs_buh','cs_ebu','cs_ebr'
]

prepath = 't4'

In [2]:
# local postgres connection only
import hidden
sql_string = hidden.psycopg2(hidden.secrets())
print('PostgreSQL connection data taken from hidden.py')

# Make the connection and cursor
conn = psycopg2.connect(sql_string, connect_timeout=3)

PostgreSQL connection data taken from hidden.py


In [3]:
# These are only used for caching
import hashlib
import pickle

def get_batch_data(base_coin, quote_coin, start_time, end_time, columns, batch_size=30000, extra_rows=0, use_cache=True):
    column_info = [(x,) + re.match('^(?P<feature>[a-z][a-z0-9]*(?:_[a-z][a-z0-9]*)*)(?:_(?P<shift>[0-9]{1,3}))?$',x).groups() for x in columns]
    max_lookback = max([(0 if x==None else int(x)) for _,_,x in column_info])
    s = base_coin+quote_coin+f"{start_time}"+f"{end_time}"+"".join(columns)+str(batch_size)+str(extra_rows)
    h = hashlib.md5(s.encode('utf-8')).hexdigest()
    should_use_cache = use_cache and (datetime.strptime(end_time, '%Y-%m-%d') < datetime.now(pytz.timezone('UTC')).replace(tzinfo=None))
    if should_use_cache:
        # Can use cache
        try:
            with open(f'../data/{prepath}/{h}.pkl', 'rb') as fp:
                print(f"Using cache file: ../data/{prepath}/{h}.pkl")
                return pickle.load(fp)
        except:
            print(f"No cache found")
            pass

    sql = f"""
select
    f.*, open_time, open, high, low, close, volume, close_time, quote_asset_volume, number_of_trades, taker_buy_base_asset_volume, taker_buy_quote_asset_volume	
from
    (
        (select * from (select id as the_pair from pairs p where p.coin1='{base_coin}' and p.coin2='{quote_coin}') z inner join candlestick_15m on the_pair=pair_id where close_time notnull and open_time < '{start_time}' order by open_time desc limit {max_lookback + extra_rows})
            union all
        (select * from (select id as the_pair from pairs p where p.coin1='{base_coin}' and p.coin2='{quote_coin}') z inner join candlestick_15m on the_pair=pair_id where close_time notnull and open_time between '{start_time}' and '{end_time}' order by open_time limit {batch_size})
    ) cm
inner join 
    features f on f.pair_id = cm.pair_id and f.candle_open_time = cm.open_time
order by
    open_time desc
"""
    base_df = pd.read_sql_query(sql, conn)
    df = base_df[['candle_open_time']].copy()
    for name, feature, shift in column_info:
        assert feature in VALID_FEATURES, f"Invalid feature: {feature} for {name}"
        df[name] = base_df[feature].shift((0 if shift==None else -int(shift)))
        
    if extra_rows == 0:
        extra_df = None
    else:
        extra_df = df.copy()
        extra_df['is_extra'] = ~extra_df['candle_open_time'].between(start_time, end_time)
        extra_df = extra_df.set_index('candle_open_time').sort_index()
        
    df = df[df['candle_open_time'].between(start_time, end_time)]
    df = df.set_index('candle_open_time').sort_index()

    ref_df = base_df[['open_time','open', 'high', 'low', 'close']].copy()
    ref_df = ref_df[ref_df['open_time'].between(start_time, end_time)]
    ref_df = ref_df.set_index('open_time').sort_index()

    batch_close_time = base_df['close_time'].max()
    
    if should_use_cache:
        print(f"Saving cache to: ../data/{prepath}/{h}.pkl")
        with open(f'../data/{prepath}/{h}.pkl', 'wb') as fp:
            pickle.dump((df, ref_df, extra_df, batch_close_time), fp, protocol=4)

    return df, ref_df, extra_df, batch_close_time

In [4]:
static_columns = ['open']
repeat_columns = ['high', 'low', 'close', 'rsi', 'trend_up3','trend_up14', 
                  'cs_ss','cs_ssr','cs_hm','cs_hmr','cs_ebu','cs_ebr']

columns = static_columns + [f"{rc}_{i}" for rc in repeat_columns for i in range(0,24)]

In [5]:
#columns

In [6]:
#len(repeat_columns)

In [7]:
mapping = {
    'ETHBTC':0,
    'BTCUSDT':1,
    'ETHUSDT':2,
    'BTCETH':-1,
    'USDTBTC':-2,
    'USDTETH':-3
}

In [8]:
from itertools import permutations

# download raw data pre-processing, this will be the same data that the simulator receives

for a,b in permutations(['ETH','BTC','USDT'],2):
    df, ref_df, _, _ = get_batch_data(a, b, '2018-01-01', '2021-08-07', columns, 500000, 0, True)
    df = df.astype(float)
    
    with open(f'../data/{prepath}_{a}_{b}.pkl', 'wb') as fp:
        pickle.dump(df, fp, protocol=4)

Using cache file: ../data/t4/18286b0cdf17618dcef3abe64553d1c3.pkl
Using cache file: ../data/t4/a982a909803ec8b7ca4388a04d3c245c.pkl
Using cache file: ../data/t4/8c4308c7bcf0dd7b87615dddc7959f43.pkl
Using cache file: ../data/t4/82c7aca7ca8c83fea53c3f75cea04ded.pkl
Using cache file: ../data/t4/eb955af59f73f18597e288971c1e1b95.pkl
Using cache file: ../data/t4/b4a7412a0032e36e55f96287d222edb2.pkl


In [9]:
# additional ETL here, like normalization, scaling and other stuff, also create rows for target use

dfs = []

for a,b in permutations(['ETH','BTC','USDT'],2):
    with open(f'../data/{prepath}_{a}_{b}.pkl', 'rb') as fp:
        df = pickle.load(fp)
        df = df.astype(float)
    
        for c in df.columns:
            if c.startswith("trend_up"):
                d = 'tc2x_' + c.replace('_up','')
                df[d] = (df[c] - 0.5) * 2
            elif c.startswith("rsi_"):
                df['tc2x_'+c] = (df[c] - 50) / 50
            elif c.startswith("high_") or c.startswith("low_") or c.startswith("close_"):
                df['tc2x_'+c] = ((df[c] / df['open']) - 1) * 30
        
        flag_pairs = [('tc2x_ss','cs_ss','cs_ssr'),('tc2x_hm','cs_hm','cs_hmr'),('tc2x_eb','cs_ebu','cs_ebr')]
        
        for newp,p1,p2 in flag_pairs:
             for i in range(0,24):
                    df[f"{newp}_{i}"] = df[f"{p1}_{i}"] - df[f"{p2}_{i}"]
        
        shifts = []
        for i in range(1,5):
            shifts.append(((df['close_0'].shift(-i) / df['open']) - 1))
            #df[f'tc2y_close_{i}'] =((df['close_0'].shift(-i) / df['open']) - 1) * 30
        shift_df = pd.concat(shifts, axis=1).dropna()
        
        df = df[[c for c in df.columns if c.startswith("tc2")]]

        df['y'] = (shift_df < 0).astype(int).sum(axis=1) / 4
        df['pair_id'] = mapping[f"{a}{b}"]
        df = df.reset_index().set_index(['pair_id','candle_open_time']).dropna()
        
        dfs.append(df)

df = pd.concat(dfs)
dfs = []

In [10]:
df

tc2x_high_0  tc2x_high_1  tc2x_high_2  \
pair_id candle_open_time                                             
 0      2018-01-01 00:00:00     0.163476     0.038070    -0.013996   
        2018-01-01 00:15:00     0.098580     0.007240    -0.117516   
        2018-01-01 00:30:00     0.117209     0.162977     0.071442   
        2018-01-01 00:45:00     0.071463     0.126177     0.171958   
        2018-01-01 01:00:00     0.076354     0.018949     0.073568   
...                                  ...          ...          ...   
-2      2021-08-06 22:00:00     0.000000     0.025456    -0.076368   
        2021-08-06 22:15:00     0.000000     0.127823     0.153387   
        2021-08-06 22:30:00     0.076825     0.051216     0.179257   
        2021-08-06 22:45:00     0.051173     0.051173     0.025586   
        2021-08-06 23:00:00     0.076989     0.141146     0.141146   

                             tc2x_high_3  tc2x_high_4  tc2x_high_5  \
pair_id candle_open_time                                             
 0      2018-01-01 00:00:00    -0.106931    -0.132124    -0.091255   
        2018-01-01 00:15:00    -0.169312    -0.261766    -0.286828   
        2018-01-01 00:30:00    -0.053581    -0.105488    -0.198140   
        2018-01-01 00:45:00     0.080396    -0.044664    -0.096587   
        2018-01-01 01:00:00     0.119269     0.027867    -0.096976   
...                                  ...          ...          ...   
-2      2021-08-06 22:00:00    -0.038184    -0.101824    -0.127280   
        2021-08-06 22:15:00     0.051129     0.089476     0.025565   
        2021-08-06 22:30:00     0.204866     0.102433     0.140845   
        2021-08-06 22:45:00     0.153518     0.179104     0.076759   
        2021-08-06 23:00:00     0.115483     0.243798     0.269461   

                             tc2x_high_6  tc2x_high_7  tc2x_high_8  \
pair_id candle_open_time                                             
 0      2018-01-01 00:00:00    -0.216101    -0.216101    -0.271526   
        2018-01-01 00:15:00    -0.246171    -0.370370    -0.370370   
        2018-01-01 00:30:00    -0.223256    -0.182512    -0.306977   
        2018-01-01 00:45:00    -0.189266    -0.214389    -0.173633   
        2018-01-01 01:00:00    -0.148807    -0.241324    -0.266404   
...                                  ...          ...          ...   
-2      2021-08-06 22:00:00    -0.140008    -0.012728    -0.241833   
        2021-08-06 22:15:00     0.000000    -0.012782     0.115040   
        2021-08-06 22:30:00     0.076825     0.051216     0.038412   
        2021-08-06 22:45:00     0.115139     0.051173     0.025586   
        2021-08-06 23:00:00     0.166809     0.205304     0.141146   

                             tc2x_high_9  tc2x_high_10  tc2x_high_11  \
pair_id candle_open_time                                               
 0      2018-01-01 00:00:00    -0.300638     -0.292800     -0.363901   
        2018-01-01 00:15:00    -0.425508     -0.454470     -0.446672   
        2018-01-01 00:30:00    -0.306977     -0.362233     -0.391256   
        2018-01-01 00:45:00    -0.298135     -0.298135     -0.353408   
        2018-01-01 01:00:00    -0.225719     -0.350004     -0.350004   
...                                  ...           ...           ...   
-2      2021-08-06 22:00:00    -0.216377     -0.280017     -0.241833   
        2021-08-06 22:15:00    -0.115040     -0.089476     -0.153387   
        2021-08-06 22:30:00     0.166453     -0.064020     -0.038412   
        2021-08-06 22:45:00     0.012793      0.140725     -0.089552   
        2021-08-06 23:00:00     0.115483      0.102652      0.230967   

                             tc2x_high_12  tc2x_high_13  tc2x_high_14  \
pair_id candle_open_time                                                
 0      2018-01-01 00:00:00     -0.325831     -0.367260     -0.218900   
        2018-01-01 00:15:00     -0.517405     -0.479532     -0.520746   
        2018-01-01 00:30:00     -0.383442     -0.454326     -0.4163

In [11]:
df['y'].quantile(np.linspace(0.0, 1.0, num=21))

0.00    0.00
0.05    0.00
0.10    0.00
0.15    0.00
0.20    0.00
0.25    0.00
0.30    0.00
0.35    0.25
0.40    0.25
0.45    0.50
0.50    0.50
0.55    0.50
0.60    0.75
0.65    0.75
0.70    1.00
0.75    1.00
0.80    1.00
0.85    1.00
0.90    1.00
0.95    1.00
1.00    1.00
Name: y, dtype: float64

In [12]:
with open(f'../data/{prepath}/full.pkl', 'wb') as fp:
    pickle.dump(df, fp, protocol=4)

In [13]:
with open(f'../data/{prepath}/train.pkl', 'wb') as fp:
    pickle.dump((df[df.index.get_level_values(1) < '2021-01-01']).reset_index(drop=True), fp, protocol=4)

In [14]:
with open(f'../data/{prepath}/test.pkl', 'wb') as fp:
    pickle.dump((df[df.index.get_level_values(1) >= '2021-01-01']).reset_index(drop=True), fp, protocol=4)

In [15]:
df['y'].value_counts()

0.00    233065
1.00    229130
0.25    100488
0.75    100177
0.50     91045
Name: y, dtype: int64